In [5]:
# Import some libraries

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

import os
import pickle
import numpy as np

from skimage import io

from matplotlib import pyplot as plt
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['figure.figsize'] = 15, 25

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
input_dim = (3, 256, 306)

In [32]:
import ConvAE_AutoConf as ConvAE

feature_maps = 96
depth = 10
pooling_freq = 2 # large number to disable pooling layers
batch_norm_freq = 2
strided_conv_freq = 6
strided_conv_feature_maps = 32
code_size = 8

CONV_ENC_BLOCK = [("conv1", feature_maps), ("relu1", None)]
CONV_ENC_LAYERS = ConvAE.create_network(
    CONV_ENC_BLOCK, depth, 
    pooling_freq=pooling_freq,
    strided_conv_freq=strided_conv_freq, 
    strided_conv_channels=strided_conv_feature_maps,
    batch_norm_freq=batch_norm_freq)

CONV_ENC_NW = CONV_ENC_LAYERS + [("flatten1", None), ("linear1", 1024), ("relu1", None), ("linear1", 256)]
model = ConvAE.ConvAE(input_dim, enc_config=CONV_ENC_NW, store_activations=False)
# print("Encoder Nw Spec:")
# CONV_ENC_NW
print(model)

ConvAE(
  (encoder): ModuleList(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): ReLU()
    (8): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13): ReLU()
    (14): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (15): ReLU(

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class DeFlatten(nn.Module):
    def __init__(self, *args):
        super(DeFlatten, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)

In [33]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(96, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            Flatten(),
            nn.Linear(in_features=1536, out_features=1024, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=256, bias=True),
        )
    def forward(self, x):
        x = self.encoder(x)
        return x
    
model = CNN().to(device)
criterion = nn.MSELoss()

In [34]:
from torchsummary import summary

summary(model, (input_dim))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 256, 306]           2,592
              ReLU-2         [-1, 96, 256, 306]               0
            Conv2d-3         [-1, 96, 256, 306]          82,944
              ReLU-4         [-1, 96, 256, 306]               0
         MaxPool2d-5         [-1, 96, 128, 153]               0
       BatchNorm2d-6         [-1, 96, 128, 153]             192
            Conv2d-7         [-1, 96, 128, 153]          82,944
              ReLU-8         [-1, 96, 128, 153]               0
            Conv2d-9         [-1, 96, 128, 153]          82,944
             ReLU-10         [-1, 96, 128, 153]               0
        MaxPool2d-11           [-1, 96, 64, 76]               0
      BatchNorm2d-12           [-1, 96, 64, 76]             192
           Conv2d-13           [-1, 96, 64, 76]          82,944
             ReLU-14           [-1, 96,

In [ ]:
from IPython.display import clear_output

num_epochs = 50
dataset_len = len(train_loader.dataset)
val_dataset_len = len(val_loader.dataset)
validation_losses = []
training_losses = []
for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    total = 0
    for data in train_loader:
        img, expected_output = data
        img = img.to(device)
        # ===================forward=====================
        output = model(img) 
        loss = criterion(output, expected_output)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        clear_output(wait=True)
        total += len(data[0])     
        if len(validation_losses) == 0:
            print(f'epoch [{epoch + 1}/{num_epochs}], data trained:{100 * total / dataset_len :.3f}%, training loss:{loss.item():.4f}')
        else:
            print(f'epoch [{epoch + 1}/{num_epochs}], data trained:{100 * total / dataset_len :.3f}%, training loss:{loss.item():.4f}, validation loss (prev epoch):{validation_losses[-1]}')
    
    with torch.no_grad():
        total_vloss = 0
        for val_data in val_loader:
            vimg, v_expected_output = val_data
            vimg = vimg.to(device)
            voutput = model(vimg)
            vloss = criterion(voutput, v_expected_output)
            total_vloss += vloss
        validation_losses.append(total_vloss)
        
    with torch.no_grad():
        total_tloss = 0
        for train_data in train_loader:
            timg, t_expected_output = train_data
            timg = timg.to(device)
            toutput = model(timg)
            tloss = criterion(toutput, t_expected_output)
            total_tloss += tloss
        training_losses.append(total_tloss)

    if (epoch + 1) % 10 == 0:
        torch.save(model, 'Models/gpu_model_b64_w2_e'+ str(epoch + 1) +'.pt')
        model.to(torch.device('cpu'))
        torch.save(model, 'Models/cpu_model_b64_w2_e'+ str(epoch + 1) +'.pt')
        model.to(device)